In [ ]:
import os
import numpy as np
import numpy.random as R
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Reshape, Flatten, Conv2D, Conv2DTranspose
from keras.layers import LeakyReLU, Dropout, MaxPooling2D, Embedding, Concatenate
import matplotlib.pyplot as plt
from tqdm import tqdm
import glob
import shutil
from PIL import Image
import tensorflow as tf
import random
import cv2

In [ ]:
dataset_path = '.../Dataset_BUSI_with_GT'

benign_folder = 'benign'
malignant_folder = 'malignant'
normal_folder = 'normal'

benign_path = os.path.join(dataset_path, benign_folder)
malignant_path = os.path.join(dataset_path, malignant_folder)
normal_path = os.path.join(dataset_path, normal_folder)

def move_non_mask_files(source_folder, output_folder):
    # Create a new folder to store non-mask files
    os.makedirs(output_folder, exist_ok=True)

    # Get a list of all file paths in the source folder
    file_paths = glob.glob(os.path.join(source_folder, "*"))

    # Move non-mask files to the new folder
    for file_path in file_paths:
        file_name = os.path.basename(file_path)
        if "_mask" not in file_name:
            destination_path = os.path.join(output_folder, file_name)
            shutil.move(file_path, destination_path)

    print("Non-mask files have been extracted to the 'non_mask_files' folder.")

move_non_mask_files(normal_path,output_folder='.../Dataset_BUSI_with_GT/normal_correct/')
move_non_mask_files(benign_path,output_folder='.../Dataset_BUSI_with_GT/benign_correct/')
move_non_mask_files(malignant_path,output_folder='.../Dataset_BUSI_with_GT/malignant_correct/')

def select_random_images(folder_path, n):
    image_filenames = os.listdir(folder_path)
    random.shuffle(image_filenames)
    selected_filenames = image_filenames[:n]
    return selected_filenames

normal_images = select_random_images('.../Dataset_BUSI_with_GT/normal_correct/', 10)
benign_images = select_random_images('.../Dataset_BUSI_with_GT/benign_correct/', 10)
malignant_images = select_random_images('.../Dataset_BUSI_with_GT/malignant_correct/', 10)

data_augmentation = ImageDataGenerator(
    rotation_range=30,
    horizontal_flip=True,
    vertical_flip=True,
    # elastic_deformation=True
)

for image_filename in benign_images + malignant_images:
    if image_filename in benign_images:
        class_name = 'benign'
        class_path = '.../Dataset_BUSI_with_GT/benign_correct/'
    else:
        class_name = 'malignant'
        class_path = '.../Dataset_BUSI_with_GT/malignant_correct/'

    # load image
    image_path = os.path.join(class_path, image_filename)
    image = cv2.imread(image_path)

    # data augmentation
    augmented_images = data_augmentation.flow(np.expand_dims(image, axis=0), batch_size=1)
    augmented_image = next(augmented_images)[0].astype(np.uint8)

In [ ]:
def generate_real_samples():
    pass

def build_generator(latent_dim, n_classes=3):
    in_label = Input(shape=(1,))

    #Class-Label embedding
    label = Embedding(n_classes, 50)(in_label)
    n_nodes = 7 * 7

    #Expanding class-label embedding
    label = Dense(n_nodes)(label)

    #Converting flat array as 2d image 
    label = Reshape((7,7, 1))(label)

    in_latent = Input(shape=(latent_dim,))
    n_nodes = 128 * 7 * 7

    #expanding random noise vector and converting in 7x7x128 image
    gen = Dense(n_nodes)(in_latent)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Reshape((7, 7, 128))(gen)

    #Adding class-label 2d image to this random noise image
    merge = Concatenate()([gen, label])

    #Creating features in image by upsampling 
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(merge)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
    gen = LeakyReLU(alpha=0.2)(gen)

    #Converting final image to a 1 channel image
    out_layer = Conv2D(1, (7,7), activation='tanh', padding='same')(gen)
    model = Model([in_latent, in_label], out_layer)
    return model

generator = build_generator(latent_dim)
generator.summary()

In [ ]:
def generate_latent_vector():
    pass

def generate_fake_samples():
    pass

In [ ]:
def build_discriminator(in_shape=(28,28,1), n_classes=3):
    in_label = Input(shape=(1,))
    label = Embedding(n_classes, 50)(in_label)
    n_nodes = in_shape[0] * in_shape[1]
    label = Dense(n_nodes)(label)
    label = Reshape((in_shape[0], in_shape[1], 1))(label)
    in_image = Input(shape=in_shape)
    merge = Concatenate()([in_image, label])
    disc = Conv2D(128, (3,3), strides=(2,2), padding='same')(merge)
    disc = LeakyReLU(alpha=0.2)(disc)
    disc = Conv2D(128, (3,3), strides=(2,2), padding='same')(disc)
    disc = LeakyReLU(alpha=0.2)(disc)
    disc = Flatten()(disc)
    disc = Dropout(0.4)(disc)
    out_layer = Dense(1, activation='sigmoid')(disc)
    model = Model([in_image, in_label], out_layer)
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

discriminator = build_discriminator()
discriminator.summary()

In [ ]:
def build_cgan():
    pass